## Описание

Здесь всё то же самое, что с Симпсонами, но с аугментацией данных случайными поворотами.

In [ ]:
import os
from pathlib import Path

import numpy as np
from numpy.random import randint
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.measure import block_reduce

from scipy import ndimage
from scipy.ndimage.filters import median_filter

from keras import Model
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dense
from keras.layers import Multiply
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import BatchNormalization as BatchNorm

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

os.chdir("/content/drive/My Drive")

In [ ]:
input_size = 32
input_gray = False
n_input_channels = 1 if input_gray else 3
input_shape = (input_size,) * 2 + (n_input_channels,)
dataset_root = Path(".")

In [ ]:
def load_img(img_path):
    img = imread(str(img_path), as_gray=input_gray)
    img = img.reshape(img.shape[:-1] + (n_input_channels,))
    img = img.astype(np.float32)

    if not input_gray:
        img /= 255

    max_side = max(img.shape)

    if max_side > input_size:
        block_size = (max_side + input_size - 1) // input_size
        img = block_reduce(img, block_size=(block_size,) * 2 + (1,), func=np.median)

    dy = (input_size - img.shape[0]) // 2
    dx = (input_size - img.shape[1]) // 2
    img_sized = np.zeros(input_shape, dtype=np.float32)
    img_sized[dy:img.shape[0]+dy, dx:img.shape[1]+dx] = img
    img = img_sized
    
    img = median_filter(img, 3)
    
    m = img.min()
    M = img.max()
    img = (img - m) / (M - m)
    
    return img


def join_paths(*args):
    return Path.joinpath(*(Path(arg) for arg in args))

In [ ]:
dataset_df["label"].unique()

In [ ]:
dataset_df = pd.read_csv("train.csv")

labels = dataset_df["label"].unique()

idx_by_label = {
    label: i
    for i, label in enumerate(labels)
}

label_min = 500

erase_size_min = 5
erase_size_max = 8

dataset_x = []
dataset_idx = []
dataset_y = []

count_by_label_idx = [0] * len(labels)

for i, row in dataset_df.iterrows():
    dataset_x.append(load_img(join_paths(dataset_root, "data", row["file_name"])))
    
    label_idx = idx_by_label[row["label"]]
    dataset_idx.append(label_idx)
    dataset_y.append(to_categorical(label_idx, num_classes=len(labels)))
    count_by_label_idx[label_idx] += 1
    
for i in range(len(dataset_x)):
    n = count_by_label_idx[dataset_idx[i]]
    
    if n >= label_min:
        continue
    
    mul_factor = (label_min + n - 1) // n
    
    for j in range(mul_factor - 1):
        img = dataset_x[i]
        
        angle = randint(-10, 11)
        
        img = ndimage.rotate(img, angle, reshape=False)
        
        m = img.min()
        M = img.max()
        img = (img - m) / (M - m)
        
#         erase_x1 = randint(input_size - erase_size_min)
#         erase_y1 = randint(input_size - erase_size_min)

#         erase_x2 = randint(
#             erase_x1 + erase_size_min,
#             min(input_size, erase_x1 + erase_size_max) + 1
#         )
        
#         erase_y2 = randint(
#             erase_y1 + erase_size_min,
#             min(input_size, erase_y1 + erase_size_max) + 1
#         )
        
#         img[erase_y1:erase_y2, erase_x1:erase_x2] = \
#             np.random.rand(erase_y2 - erase_y1, erase_x2 - erase_x1, n_input_channels)
        
        dataset_x.append(img)
        dataset_y.append(dataset_y[i])

dataset_x = np.array(dataset_x)
dataset_y = np.array(dataset_y)

In [ ]:
plt.imshow(dataset_x[-])

In [ ]:
def save_array(name):
    np.save(f"{name}.npy", globals()[name])


def save_arrays(*args):
    for arg in args:
        save_array(arg)

        
def load_array(name):
    globals()[name] = np.load(f"{name}.npy")

    
def load_arrays(*args):
    for arg in args:
        load_array(arg)

In [ ]:
save_arrays("dataset_x", "dataset_y")

In [ ]:
load_arrays("dataset_x", "dataset_y")

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(dataset_x, dataset_y, test_size=0.1)

In [ ]:
save_arrays("train_x", "test_x", "train_y", "test_y")

In [ ]:
load_arrays("train_x", "test_x", "train_y", "test_y")

In [ ]:
with open("labels.txt", "w") as f:
    f.write(" ".join(labels))

In [ ]:
with open("labels.txt", "r") as f:
    labels = f.read().split()

In [ ]:
batch_size = 32
n_epochs = 50

use_global_avg_pooling = True
use_se_block = False

In [ ]:
act = "elu"

model_input = Input(shape=input_shape)

def block(x, k):
    x = Conv2D(k, 3, padding="same", activation=act)(x)
    x = Conv2D(k, 3, activation=act)(x)
    x = MaxPooling2D(2)(x)
    x = Dropout(0.25)(x)
    x = BatchNorm()(x)
    return x

def se_block(x, n):
    branch = x
    branch = GlobalAveragePooling2D()(branch)
    branch = Dense(8, activation=act)(branch)
    branch = Dense(n, activation="sigmoid")(branch)
    
    return Multiply()([x, branch])

model = model_input
model = block(model, 32)
model = block(model, 64)

if use_se_block:
    model = se_block(model, 64)

model = block(model, 64)

if use_global_avg_pooling:
    model = GlobalAveragePooling2D()(model)
else:
    model = Flatten()(model)
    model = Dense(512, activation=act)(model)
    model = Dropout(0.5)(model)

model = Dense(len(labels), activation="softmax")(model)
model = Model(inputs=model_input, outputs=model)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"]
)

model.summary()

In [ ]:
mcp_save = ModelCheckpoint(
    "model.hdf5", 
    verbose=1,
    save_best_only=True
)

model.fit(
    train_x, train_y,
    validation_data=(test_x, test_y),
    epochs=n_epochs,
    batch_size=batch_size,
    callbacks=[mcp_save]
)

In [ ]:
model = load_model("model.hdf5")

model.summary()

In [ ]:
task_df = pd.read_csv("test.csv")

task_x = np.zeros((len(task_df),) + input_shape)

for i, row in task_df.iterrows():
    task_x[i] = load_img(join_paths(dataset_root, "data", row["file_name"]))

In [ ]:
task_y = model.predict(task_x, batch_size=batch_size)

In [ ]:
ans_df = pd.DataFrame()

ans_df["label"] = [labels[v.argmax()] for v in task_y]

ans_df.index = task_df["file_name"]
ans_df.index.name = "file_name"

In [ ]:
i = 200
print(ans_df.iloc[i]["label"])
plt.imshow(task_x[i])
# pd.read_csv("train.csv")["label"].value_counts()

In [ ]:
ans_df.to_csv("ans.csv")